In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Importamos el dataset**

In [0]:
import pandas as pd
ratings = pd.read_csv("/content/drive/My Drive/ML/myanimelist/animelists_cleaned.csv")

In [0]:
users= pd.read_csv("/content/drive/My Drive/ML/myanimelist/users_cleaned.csv")

In [0]:
animes= pd.read_csv("/content/drive/My Drive/ML/myanimelist/anime_cleaned.csv")

# Analizamos el dataset

In [6]:
print("tamaño de usuarios: ", users.shape)
print("tamaño de animes: ", animes.shape)
print("tamaño de usuariosXanimes: ", ratings.shape)

tamaño de usuarios:  (108711, 17)
tamaño de animes:  (6668, 33)
tamaño de usuariosXanimes:  (31284030, 11)


In [7]:
users.head(2)

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,NaN,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,NaN,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903


In [8]:
pd.options.display.max_columns = None
#display(animes.head())
print(animes.title.to_string(index=False))

Inu x Boku SS
                                 Seto no Hanayome
                               Shugo Chara!! Doki
                                    Princess Tutu
                              Bakuman. 3rd Season
                              Yume-iro Pâtissière
                                     Ultra Maniac
                    Shakugan no Shana II (Second)
                      Nodame Cantabile: Paris-hen
                           Ouran Koukou Host Club
                               Junjou Romantica 2
                            Kaichou wa Maid-sama!
                             Sekaiichi Hatsukoi 2
                           Tonari no Kaibutsu-kun
                                           Bleach
                                          Chobits
                                   Kimi ni Todoke
                               Naruto: Shippuuden
                                          Ranma ½
                                        Toradora!
                              Bakuma

In [9]:
ratings.head(2)

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN


In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108711 entries, 0 to 108710
Data columns (total 17 columns):
username                    108710 non-null object
user_id                     108711 non-null int64
user_watching               108711 non-null int64
user_completed              108711 non-null int64
user_onhold                 108711 non-null int64
user_dropped                108711 non-null int64
user_plantowatch            108711 non-null int64
user_days_spent_watching    108711 non-null float64
gender                      108711 non-null object
location                    108706 non-null object
birth_date                  108711 non-null object
access_rank                 0 non-null float64
join_date                   108711 non-null object
last_online                 108711 non-null object
stats_mean_score            108711 non-null float64
stats_rewatched             108711 non-null float64
stats_episodes              108711 non-null int64
dtypes: float64(4), int64(7), 

# Sistema de Recomendacion (Content Based Filtering)

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
#from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [0]:
ratings = pd.merge(ratings, users[['username', 'user_id']], how='left', on='username')

In [13]:
 #Esto es solo  para poder hacer el ctrl + d como en sublime, obviar esta parte.
 # Print the location of Jupyter's config directory
 from jupyter_core.paths import jupyter_config_dir
 jupyter_dir = jupyter_config_dir()
 print(jupyter_dir)

 # Print the location of custom.js
 import os.path
 custom_js_path = os.path.join(jupyter_dir, 'custom', 'custom.js')
 print(custom_js_path)

 # Print the contents of custom.js, if it exists.
 if os.path.isfile(custom_js_path):
     with open(custom_js_path) as f:
         print(f.read())
 else:
     print("You don't have a custom.js file")

/root/.jupyter
/root/.jupyter/custom/custom.js
You don't have a custom.js file


In [0]:
#El notebook consta de 4 partes, la 1 se analizará la data, teniendo como resultados los mejores animes más influyentes, tanto
#históricos como por género.
#La segunda constará de una recomendación basada en la descripción del anime
#La tercera constará de una recomendación basada en el productor, studio y género del anime
#La cuarta constará de una recomendación similar a la anterior pero tomando en cuenta el ranking de las puntuaciones

In [15]:
#Esta es la primera parte
vote_counts = animes[animes['scored_by'].notnull()]['scored_by'].astype('int')
vote_averages = animes[animes['score'].notnull()]['score'].astype('int')
C = vote_averages.mean()
C

6.357228554289142

In [16]:
m = vote_counts.quantile(0.95)
m

112875.44999999994

In [17]:
qualified = animes[(animes['scored_by'] >= m) & (animes['scored_by'].notnull()) & (animes['score'].notnull())][['title', 'aired_from_year', 'scored_by', 'score', 'popularity', 'genre']]
qualified['scored_by'] = qualified['scored_by'].astype('int')
qualified['score'] = qualified['score'].astype('int')
qualified.shape

(334, 6)

In [0]:
def weighted_rating(x):
    v = x['scored_by']
    R = x['score']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [0]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [0]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [21]:
qualified.head(15)


,title,aired_from_year,scored_by,score,popularity,genre,wr
1261,Fullmetal Alchemist: Brotherhood,2009.0,733592,9,4,"Action, Military, Adventure, Comedy, Drama, Ma...",8.647590
1475,Steins;Gate,2011.0,563857,9,8,"Thriller, Sci-Fi",8.559199
529,Kimi no Na wa.,2016.0,471398,9,33,"Supernatural, Drama, Romance, School",8.489445
223,Hunter x Hunter (2011),2011.0,403377,9,35,"Action, Adventure, Super Power, Shounen",8.422174
1366,Clannad: After Story,2008.0,363239,9,55,"Slice of Life, Comedy, Supernatural, Drama, Ro...",8.373462
3156,Koe no Katachi,2016.0,296261,9,77,"Drama, School, Shounen",8.270894
3802,Death Note,2006.0,1009477,8,1,"Mystery, Police, Psychological, Supernatural, ...",7.834786
3147,Gintama,2006.0,141830,9,95,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...",7.828828
3897,Shingeki no Kyojin,2013.0,940211,8,2,"Action, Military, Mystery, Super Power, Drama,...",7.823919
4215,One Punch Man,2015.0,691845,8,5,"Action, Sci-Fi, Comedy, Parody, Super Power, S...",7.769574


In [0]:
s = animes.apply(lambda x: pd.Series(x['genre']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_animes = animes.drop('genre', axis=1).join(s)

In [0]:
def build_chart(genre, percentile=0.60):
    df = gen_animes[gen_animes['genre'].astype(str).str.contains(genre)]
    scored_bys = df[df['scored_by'].notnull()]['scored_by'].astype('int')
    scores = df[df['score'].notnull()]['score'].astype('int')
    C = scores.mean()
    m = scored_bys.quantile(percentile)
    
    qualified = df[(df['scored_by'] >= m) & (df['scored_by'].notnull()) & (df['score'].notnull())][['title', 'aired_from_year', 'scored_by', 'score', 'popularity']]
    qualified['scored_by'] = qualified['scored_by'].astype('int')
    qualified['score'] = qualified['score'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['scored_by']/(x['scored_by']+m) * x['score']) + (m/(m+x['scored_by']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [24]:
#con esto saco los mejores animes por género.
build_chart('Action, Shounen')

,title,aired_from_year,scored_by,score,popularity,wr
5004,Mirai Nikki,2010.0,81640,7,562,6.898421
3405,Captain Tsubasa,1983.0,28285,7,1600,6.774660
5274,Captain Tsubasa: Road to 2002,2001.0,17252,7,2243,6.698767
5878,Monochrome Factor,2008.0,15507,7,1709,6.681818
5408,Big Order (TV),2016.0,58692,5,689,5.284989


In [25]:
gen_animes.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,opening_theme,ending_theme,duration_min,aired_from_year,genre
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727.0,366,204003,2579,NaN,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0,"Comedy, Parody, Romance, School, Shounen"
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508.0,1173,70127,802,NaN,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0,"Comedy, Magic, School, Shoujo"
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,False,"Aug 16, 2002 to May 23, 2003","{'from': '2002-08-16', 'to': '2003-05-23'}",16 min. per ep.,PG-13 - Teens 13 or older,8.21,36501,307.0,916,93312,3344,Princess Tutu aired in two parts. The first pa...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,False,"Oct 6, 2012 to Mar 30, 2013","{'from': '2012-10-06', 'to': '2013-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,8.67,107767,50.0,426,182765,2082,NaN,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0,"Comedy, Drama, Romance, Shounen"


In [0]:
#Esta es la segunda parte
animes['background'] = animes['background'].fillna('')
animes['description'] = animes['background'] + animes['related']
animes['description'] = animes['description'].fillna('')

In [0]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(animes['description'])

In [28]:
tfidf_matrix.shape

(6668, 104540)

In [0]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_sim[0]

array([1.        , 0.02248097, 0.02180296, ..., 0.00797501, 0.        ,
       0.01191052])

In [0]:
animes = animes.reset_index()
titles = animes['title']
indices = pd.Series(animes.index, index=animes['title'])

In [0]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return animes.iloc[movie_indices][['title', 'scored_by', 'score', 'aired_from_year', 'genre']]


In [33]:
#En este punto tengo las recomendaciones basadas en el background del anime y el campo related.
get_recommendations('Sword Art Online').head(20)

,title,scored_by,score,aired_from_year,genre
2538,Sword Art Online II,531486,7.20,2014.0,"Action, Game, Adventure, Romance, Fantasy"
4341,Sword Art Online: Extra Edition,103446,6.88,2014.0,"Action, Adventure, Fantasy, Game, Romance"
4384,Sword Art Online: Sword Art Offline,28319,6.38,2012.0,Comedy
3014,Sword Art Online Movie: Ordinal Scale,113031,8.09,2017.0,"Action, Game, Adventure, Romance, Fantasy"
6244,Sword Art Online II: Sword Art Offline II,11421,6.75,2014.0,Comedy
633,Sword Art Online Alternative: Gun Gale Online,21075,7.30,2018.0,"Action, Game, Military, Sci-Fi, Fantasy"
3804,Sword Art Online II: Debriefing,31111,6.63,2014.0,"Action, Adventure, Fantasy, Game"
5949,Sword Gai: The Animation,5216,6.15,2018.0,"Action, Seinen, Supernatural"
3261,Sword Gai,302,5.79,2013.0,"Action, Seinen, Supernatural"
3713,Sakamichi no Apollon,93986,8.44,2012.0,"Drama, Josei, Music, Romance, School"


In [0]:
from ast import *
#Esta es la tercera parte. Ahora vamos a mejorarlo basándonos en genre, producer, studio

In [0]:
animes['producer'] = animes['producer'].fillna('')
animes['studio'] = animes['studio'].fillna('')
for index, row in animes.iterrows():
    a = row['producer'].split(',')
    a = str(a)
    a = literal_eval(a)
animes['studio'] = animes['studio'].fillna('')
for index, row in animes.iterrows():
    a = row['studio'].split(',')
    a = str(a)
    a = literal_eval(a)
animes['producer_size'] = animes['producer'].apply(lambda x: len(x))
animes['studio_size'] = animes['studio'].apply(lambda x: len(x))

In [0]:
animes['soup'] = animes['producer'] + animes['studio'] + animes['genre']


In [37]:
animes.head()

,index,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year,description,producer_size,studio_size,soup
0,0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0,Inu x Boku SS was licensed by Sentai Filmworks...,92,16,"Aniplex, Square Enix, Mainichi Broadcasting Sy..."
1,1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727.0,366,204003,2579,,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...",33,5,"TV Tokyo, AIC, Square Enix, SotsuGonzoComedy, ..."
2,2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508.0,1173,70127,802,,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...",15,9,"TV Tokyo, SotsuSatelightComedy, Magic, School,..."
3,3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,False,"Aug 16, 2002 to May 23, 2003","{'from': '2002-08-16', 'to': '2003-05-23'}",16 min. per ep.,PG-13 - Teens 13 or older,8.21,36501,307.0,916,93312,3344,Princess Tutu aired in two parts. The first pa...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0,Princess Tutu aired in two parts. The first pa...,34,14,"Memory-Tech, GANSIS, Marvelous AQLHal Film Mak..."
4,4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,False,"Oct 6, 2012 to Mar 30, 2013","{'from': '2012-10-06', 'to': '2013-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,8.67,107767,50.0,426,182765,2082,,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...",13,9,"NHK, ShueishaJ.C.StaffComedy, Drama, Romance, ..."


In [0]:
animes['soup'] = animes['soup'].fillna('')
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(animes['soup'])

In [0]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [0]:
animes = animes.reset_index()
titles = animes['title']
indices = pd.Series(animes.index, index=animes['title'])

In [42]:
#Esto estará ordenado por la similitud entre elementos.
get_recommendations('Orange').head(20)

,title,scored_by,score,aired_from_year,genre
3348,Orange: Mirai,10906,7.54,2016.0,"Sci-Fi, Drama, Romance, School, Shoujo"
6018,Mikakunin de Shinkoukei,92924,7.54,2014.0,"Slice of Life, Comedy, Romance, School"
2386,Mikakunin de Shinkoukei: Kamoniku tte Midori-p...,16059,7.19,2014.0,"Slice of Life, Comedy, Romance, Seinen"
2617,Ushinawareta Mirai wo Motomete,21389,6.83,2014.0,"Sci-Fi, Drama, Romance, School"
3142,A-Girl,369,5.48,1993.0,"Music, Romance, School, Shoujo"
6335,Ajimu: Kaigan Monogatari,4583,6.44,2001.0,"Comedy, Drama, Romance, School"
6574,Nijiiro☆Prism Girl,912,6.31,2013.0,"Comedy, Romance, School, Shoujo"
1694,Sansha Sanyou,13843,7.18,2016.0,"Slice of Life, Comedy, School"
3654,Ikemen Sengoku: Toki wo Kakeru ga Koi wa Hajim...,1600,6.23,2017.0,"Historical, Romance"
1044,Good Morning Call,2508,6.22,2001.0,"Comedy, Drama, Romance, School, Shoujo"


In [0]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = animes.iloc[movie_indices][['title', 'scored_by', 'score', 'aired_from_year', 'genre']]
    scored_bys = movies[movies['scored_by'].notnull()]['scored_by'].astype('int')
    scores = movies[movies['score'].notnull()]['score'].astype('int')
    C = scores.mean()
    m = scored_bys.quantile(0.60)
    qualified = movies[(movies['scored_by'] >= m) & (movies['scored_by'].notnull()) & (movies['score'].notnull())]
    qualified['scored_by'] = qualified['scored_by'].astype('int')
    qualified['score'] = qualified['score'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False)
    return qualified

In [44]:
#Esto estará ordenado por el WR
improved_recommendations('Orange')

,title,scored_by,score,aired_from_year,genre,wr
3574,Ao Haru Ride,184367,7,2014.0,"Comedy, Drama, Romance, School, Shoujo, Slice ...",6.755913
2205,Special A,113131,7,2008.0,"Comedy, Romance, School, Shoujo",6.678978
4539,"Clannad: After Story - Mou Hitotsu no Sekai, K...",110530,7,2009.0,"Drama, Romance, School",6.675240
6018,Mikakunin de Shinkoukei,92924,7,2014.0,"Slice of Life, Comedy, Romance, School",6.647457
1432,Karakai Jouzu no Takagi-san,49525,7,2018.0,"Comedy, Romance, School, Shounen, Slice of Life",6.553246
4657,Hal,37631,7,2013.0,"Sci-Fi, Drama, Romance, Shoujo",6.517940
5547,Seitokaichou ni Chuukoku,16879,7,2009.0,"Comedy, Drama, Romance, School, Yaoi",6.440843
2386,Mikakunin de Shinkoukei: Kamoniku tte Midori-p...,16059,7,2014.0,"Slice of Life, Comedy, Romance, Seinen",6.437287
4310,Hana yori Dango,14464,7,1996.0,"Drama, Romance, School, Shoujo",6.430238
2617,Ushinawareta Mirai wo Motomete,21389,6,2014.0,"Sci-Fi, Drama, Romance, School",6.300320


In [0]:
  improved_recommendations('Orange')

,title,scored_by,score,aired_from_year,wr
3574,Ao Haru Ride,184367,7,2014.0,6.755913
2205,Special A,113131,7,2008.0,6.678978
4539,"Clannad: After Story - Mou Hitotsu no Sekai, K...",110530,7,2009.0,6.675240
6018,Mikakunin de Shinkoukei,92924,7,2014.0,6.647457
1432,Karakai Jouzu no Takagi-san,49525,7,2018.0,6.553246
4657,Hal,37631,7,2013.0,6.517940
5547,Seitokaichou ni Chuukoku,16879,7,2009.0,6.440843
2386,Mikakunin de Shinkoukei: Kamoniku tte Midori-p...,16059,7,2014.0,6.437287
4310,Hana yori Dango,14464,7,1996.0,6.430238
2617,Ushinawareta Mirai wo Motomete,21389,6,2014.0,6.300320
